## Библиотеки

In [ ]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import torch
import torchvision
import torch.nn.functional as F
from torch import nn, optim
# from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models, utils
from torch.optim.lr_scheduler import MultiplicativeLR
# from pytorch_lightning.callbacks.early_stopping import EarlyStopping
# from pytorch_lightning import Trainer
# from ignite.handlers import EarlyStopping
# from pytorchtools import EarlyStopping

# import matplotlib.image as Image
from skimage import io
from PIL import Image
from os import listdir
from os.path import isfile, join
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, accuracy_score, roc_curve, auc, roc_auc_score, precision_score, recall_score
from sklearn.utils import class_weight

import matplotlib.pyplot as plt

from collections import defaultdict 

from shutil import copyfile
import zipfile

from tqdm import tqdm
# from tqdm.notebook import tqdm
from time import sleep

import warnings
warnings.filterwarnings('ignore')

## GPU

In [ ]:
GPU = True
device = "cuda" if GPU and torch.cuda.is_available() else "cpu"

print(f'Using device {device}')

## DataFrame

In [ ]:
df = pd.read_csv('../input/aerial-cactus-identification/train.csv')
df.head()

In [ ]:
df.shape

## Распределение данных

In [ ]:
cmap = plt.get_cmap('Blues')
colors = [cmap(i) for i in np.linspace(0, 0.7, df['has_cactus'].unique().shape[0])]

plt.title('Сlass distribution')
df['has_cactus'].value_counts().plot(kind='pie', figsize=(6, 6), autopct='%1.2f%%', shadow=True, colors=colors)
plt.show()

## Тестовая выборка

In [ ]:
submission = pd.read_csv('../input/aerial-cactus-identification/sample_submission.csv')
submission.head()

## Создание тренировочной и валидационных частей

In [ ]:
train_link, test_link = '../input/aerial-cactus-identification/train.zip', '../input/aerial-cactus-identification/test.zip'

In [ ]:
with zipfile.ZipFile(train_link, "r") as z:
    z.extractall(".")
    
with zipfile.ZipFile(test_link, "r") as z:
    z.extractall(".")

In [ ]:
Y_train, Y_valid = train_test_split(df, stratify=df['has_cactus'], random_state=42, test_size=0.2)

In [ ]:
Y_valid.head()

In [ ]:
current_path = './train/'
need_path = './valid/'
test_path = './test/'

try:
    os.makedirs(need_path)
except FileExistsError:
    pass

for link in Y_valid['id']:
    copyfile(current_path + link, need_path + link)

In [ ]:
len(os.listdir(current_path)), len(os.listdir(need_path)), len(os.listdir(test_path))

## Dataset Class

In [ ]:
class CactusDataset(Dataset):
    # папка, метки и преобразования
    def __init__(self, folder, labels, transform=None):
        self.transform = transform
        self.folder = folder
        self.labels = labels
    
    # размерность датасета
    def __len__(self):
#         return len(self.data)
        return self.labels.shape[0]
    
    # получить образец
    def __getitem__(self, index):
        image_path = os.path.join(self.folder, self.labels['id'].iloc[index])
        label =  self.labels['has_cactus'].iloc[index]
        
        image = Image.open(image_path).convert('RGB')
#         image = cv2.imread(image_path)
#         image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if self.transform is not None:
            image = self.transform(image)

        return image, label
    
    # аолучить среднее и отклонение
    def get_mean_std(self):
        image_all = np.array([np.array(self[ind][0]) for ind in tqdm(range(self.__len__()))])
        
        return image_all.mean(axis=(0, 1, 2)) / 255, image_all.std(axis=(0, 1, 2)) / 255
    
    # просмотреть некоторый набор, индексы которого будут переданы                         
    def view_sample(self, indices, mean, std, count=8):
        plt.figure(figsize=(count * 3, 3))

        for i, ind in enumerate(indices):
            image, label = self[ind]
            
            if self.transform is not None:
                image = image.squeeze().permute(1, 2, 0).numpy()
                image = std * image + mean
#                 image = np.clip(image, 0, 1)
            
            plt.subplot(1, count, i + 1)
            plt.imshow(image)
            plt.axis('off')
            plt.title(f'Cactus: {label == 1}')

In [ ]:
train_dataset = CactusDataset(folder=current_path, labels=Y_train)

In [ ]:
mean, std = train_dataset.get_mean_std()
mean, std

In [ ]:
# indices = np.random.choice(np.arange(self.__len__()), count, replace=False)
indices = list(range(8))

train_dataset.view_sample(indices=indices, mean=mean, std=std)

## Преобразования и создание нового набора

In [ ]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
#     transforms.Resize(50),
#     transforms.RandomAffine(5, shear=(2, 2)),
#     transforms.Resize(30),
    
    transforms.RandomRotation(10),
    transforms.CenterCrop(28),
    
#     transforms.RandomCrop(28),
    
    transforms.Pad(padding=2, padding_mode='symmetric'),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

train_dataset = CactusDataset(folder=current_path, labels=Y_train, transform=transform)
valid_dataset = CactusDataset(folder=need_path, labels=Y_valid, transform=transform)

In [ ]:
train_dataset.view_sample(indices=indices, mean=mean, std=std)

## Model Class

https://www.aiworkbox.com/lessons/how-to-define-a-convolutional-layer-in-pytorch

https://github.com/vdumoulin/conv_arithmetic/blob/master/README.md

https://wandb.ai/authors/ayusht/reports/Implementing-and-Tracking-the-Performance-of-a-CNN-in-Pytorch-An-Example--VmlldzoxNjEyMDU

https://www.aiworkbox.com/lessons/batchnorm2d-how-to-use-the-batchnorm2d-module-in-pytorch

https://stackoverflow.com/questions/53419474/using-dropout-in-pytorch-nn-dropout-vs-f-dropout

https://pytorch.org/docs/stable/generated/torch.nn.LeakyReLU.html#torch.nn.LeakyReLU

почему leaky_relu

https://www.quora.com/What-are-the-advantages-of-using-Leaky-Rectified-Linear-Units-Leaky-ReLU-over-normal-ReLU-in-deep-learning

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        # свёрточные
        # torch.Size([16, 3, 3, 3]) = torch.Size([out_channels, in_channels, kernel_size[0], kernel_size[1]])
        # in_channels, out_channels: кол-во входных и выходных каналов соответственно
        # kernel_size: размер ядра
        # padding: сколько пустых (0) пикселей добавится
        # stride: отступ
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        
        # избежать ковариантного сдвига и не насыщать функцию активации
        # num_features: кол-во признаков в пред. слое
        self.dense_1 = nn.BatchNorm2d(16)
        
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.dense_2 = nn.BatchNorm2d(32)
        
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.dense_3 = nn.BatchNorm2d(64)
        
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.dense_4 = nn.BatchNorm2d(128)
        
        # полносвязные
        # принимает на вход (batch_size, 512) и возвращает тензор (batch_size, 128). y = x * W^T + b, W.shape=(out_features, in_features), b.shape=(out_features)
        self.fc1 = nn.Linear(in_features=512, out_features=128)
        self.fc_dense1 = nn.BatchNorm1d(128)
        self.out = nn.Linear(in_features=128, out_features=2)
         
        # прореживание
        # p: вер-сть отключения нейрона
        self.d1 = nn.Dropout(0.5)
        # сигмоидная функция (логит)
        self.f = nn.Sigmoid()

        
    def forward(self, t):
        x = t
        x = self.conv1(x)
        x = self.dense_1(x)
        # LRELU = max{0, x} + neg * min{0, x}, neg == 0.01
        x = F.leaky_relu(x)
        # взятие максимума из прямоугольников (kernel_size[0], kernel_size[1])
        # stride: отступ пикселей при соседних взятиях максимума
        x = F.max_pool2d(x, kernel_size=2, stride=2)


        x = self.conv2(x)
        x = self.dense_2(x)
        x = F.leaky_relu(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2)

        x = self.conv3(x)
        x = self.dense_3(x)
        x = F.leaky_relu(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2)

        x = self.conv4(x)
        x = self.dense_4(x)
        x = F.leaky_relu(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2)

        # keras -> flatten(): сделать слой плотным (вытянуть)
        x = x.reshape(-1, 512)

        x = self.fc1(x)
        x = self.fc_dense1(x)
        x = F.leaky_relu(x)
#         x = self.d1(x)
        
        x = self.out(x)
        x = self.f(x)

        return x

## Параметры

In [ ]:
# норма обучения
lr = 0.02

# максимальное количество эпох 
epochs = 100

# размер батча
batch_size = 64

# всё в основном процессе
num_workers = 0

# словарь метрик
metrics = defaultdict(list)

# сама модель
model = Model().to(device)

# оптимизатор
optimizer = optim.Adam(model.parameters(), lr=lr)

# стратегия изменения нормы обучения
scheduler = MultiplicativeLR(optimizer, lr_lambda=lambda epoch: 0.9)

# загрузчики
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
valid_data_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

In [ ]:
train_data_loader

In [ ]:
# итератор по загрузчику
dataiter = iter(train_data_loader)

# один батч
images, labels = dataiter.next()

plt.figure(figsize=(8, 8))
plt.axis('off')
plt.title("Изображения тренировочного набора")
plt.imshow(np.transpose(utils.make_grid(images.to(device), padding=2, normalize=True).cpu(), (1, 2, 0)))
plt.show()

## Несбалансированность классов

In [ ]:
weight = class_weight.compute_class_weight('balanced', np.unique(Y_train['has_cactus']), Y_train['has_cactus'])
weight = {i : weight[i] for i in np.unique(Y_train['has_cactus'])}
weight

## Дополнительные метрики

In [ ]:
def other_metrics(y_true, y_pred, weights):
    balanced_accuracy = balanced_accuracy_score(y_true, y_pred, sample_weight=[weights[i] for i in y_true])
    accuracy = accuracy_score(y_true, y_pred)
    precision= precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_pred)
    
    return np.array([balanced_accuracy, accuracy, precision, recall, roc_auc])

## Тренировка сети

In [ ]:
# количество эпох без улучшения функции ошибки (на валидации)
early_stopping = 25

# счётчик
early_count = 0

# папка для сохранения модели
checkpoint_dirr = './checkpoint/'
try:
    os.makedirs(checkpoint_dirr)
except FileExistsError:
    pass

# начальная лучшая потеря
best_loss = np.inf

# матрики
balanced_accuracy, accuracy, precision, recall, roc_auc = [0] * 5

# по каждой эпохе
for epoch in range(epochs):
    # шаг изменения learning rate
    scheduler.step()
    
    # обнулить loss ошибку и метрики
    train_loss = 0
    metric = 0
    model.train()
    
    with tqdm(train_data_loader, unit="batch") as tepoch:
        tepoch.set_description(f"Epoch {epoch}")
        # по каждому батчу
        for i, batch in enumerate(tepoch):

            # получить изо и метки из батча
            images, labels = batch
            
            # оптимизация под устройство
            images = images.to(device)
            labels = labels.to(device) 

            # в 64 бита
            labels = labels.long()
            
            # прогнозы
            preds = model(images)
            pred = preds.argmax(dim=1)
            
            # функция потерь: https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
            loss = F.cross_entropy(preds, labels)

            # обнулить градиент перед обратным распространением
            optimizer.zero_grad()
            
            # вычислить градиент потерь
            loss.backward()
            
            # обновить параметры
            optimizer.step()
            
            # значение ошибки
            train_loss += loss.item()
            
            # другие метрики
            metric += other_metrics(labels.tolist(), pred.tolist(), weight)
            balanced_accuracy, accuracy, precision, recall, roc_auc = metric
            
            tepoch.set_postfix(loss=train_loss / (i + 1), balanced_accuracy=balanced_accuracy / (i + 1), accuracy=accuracy / (i + 1), precision=precision / (i + 1), 
                               recall=recall / (i + 1), roc_auc=roc_auc / (i + 1), lr=scheduler.get_lr()[0])
        
        # в словарь
        metrics['loss'].append(train_loss / (i + 1))
        metrics['balanced_accuracy'].append(balanced_accuracy / (i + 1))
        metrics['accuracy'].append(accuracy / (i + 1))
        metrics['precision'].append(precision / (i + 1))
        metrics['recall'].append(recall / (i + 1))
        metrics['roc_auc'].append(roc_auc / (i + 1))
    
    sleep(0.1)
    
    model.eval()
    
    # тоже самое, но без градиентов (т.к. валидация)
    with torch.no_grad():
        valid_loss = 0
        metric = 0
        with tqdm(valid_data_loader, unit="batch") as tepoch:
            tepoch.set_description(f"Epoch {epoch}")
        
            for i, valid_batch in enumerate(tepoch):
                valid_images, valid_labels = valid_batch
                valid_images = valid_images.to(device)
                valid_labels = valid_labels.to(device)
                valid_labels = valid_labels.long()

                valid_preds = model(valid_images)
                valid_pred = valid_preds.argmax(dim=1)

                loss_valid = F.cross_entropy(valid_preds ,valid_labels)
                
                valid_loss += loss_valid.item()
                
                metric += other_metrics(valid_labels.tolist(), valid_pred.tolist(), weight)
                balanced_accuracy, accuracy, precision, recall, roc_auc = metric
                
                tepoch.set_postfix(loss=valid_loss / (i + 1), balanced_accuracy=balanced_accuracy / (i + 1), accuracy=accuracy / (i + 1), precision=precision / (i + 1), 
                               recall=recall / (i + 1), roc_auc=roc_auc / (i + 1))

            metrics['val_loss'].append(valid_loss / (i + 1))
            metrics['val_balanced_accuracy'].append(balanced_accuracy / (i + 1))
            metrics['val_accuracy'].append(accuracy / (i + 1))
            metrics['val_precision'].append(precision / (i + 1))
            metrics['val_recall'].append(recall / (i + 1))
            metrics['val_roc_auc'].append(roc_auc / (i + 1))

    sleep(0.1)
    
    # если ошибка улучшилась
    if valid_loss < best_loss:
        # пересохранить
        best_loss = valid_loss
        
        # занулить счётчик
        early_count = 0
        
        # записать 
        torch.save(model.state_dict(), f'{checkpoint_dirr}epoch:{epoch}.pt')
    else:
        early_count += 1

        if early_count > early_stopping:
            print(f"Loss did not improve over {early_stopping} epochs => early stopping")
            break

## Графики

In [ ]:
def graph_plot(history, typ=False):
    if typ:
        for i in history.keys():
            print(f'{i} = [{min(history[i])}; {max(history[i])}]\n')
    
    epoch = len(history['loss'])
    # на каждую: (train, val) + lr
    size = len(history.keys()) // 2 + 1
    
    ncols = 3
    nrows = np.ceil(size / ncols)
    
    fig = plt.figure(figsize=(30, 20))
    i = 1
    for k in list(history.keys()):
        if 'val' not in k:
            fig.add_subplot(nrows, ncols, i)
            plt.plot(history[k][2:], marker='o', markersize=5)
            if k != 'lr':
                plt.plot(history['val_' + k][2:], marker='o', markersize=5)
            plt.title(k, fontsize=10)

            plt.ylabel(k)
            plt.xlabel('epoch')
            plt.grid()

            plt.yticks(fontsize=10, rotation=30)
            plt.xticks(fontsize=10, rotation=30)
            plt.legend(['train', 'valid'], loc='upper left', fontsize=10, title_fontsize=15)
            i += 1
#         plt.show()

graph_plot(metrics)

In [ ]:
np.argmax(-np.array(metrics['val_loss']))

In [ ]:
[name for name in sorted(os.listdir(checkpoint_dirr)) if str(np.argmax(-np.array(metrics['val_loss']))) in name][0]

## Загрузка лучшей

In [ ]:
model = Model().to(device)
model.load_state_dict(torch.load(f"{checkpoint_dirr}{[name for name in sorted(os.listdir(checkpoint_dirr)) if str(np.argmax(-np.array(metrics['val_loss']))) in name][0]}"))
model.eval()

## Прогноз

In [ ]:
test_dataset = CactusDataset(folder=test_path, labels=submission, 
                             transform=transforms.Compose([transforms.ToTensor(), 
                                                           transforms.Normalize(mean=mean, std=std)]))

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [ ]:
predict = []

for test_batch in test_loader:
    data, target = test_batch
    data, target = data.to(device), target.to(device)
    output = model(data)

    predict += output.argmax(dim=1).tolist()

submission['has_cactus'] = predict
submission.to_csv('submission.csv', index=False)

In [ ]:
submission.head()